In [2]:
import pandas as pd 
import numpy as np
from IPython.display import clear_output
import time
import pickle

import matplotlib.pyplot as plt
from matplotlib.cbook import flatten
import matplotlib.cm as cm
import matplotlib as matplotlib

from fastdtw import fastdtw

def color_map_color(value, cmap_name='coolwarm', vmin=0, vmax=10):
    # norm = plt.Normalize(vmin, vmax)
    norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = cm.get_cmap(cmap_name)  # PiYG
    rgb = cmap(norm(abs(value)))[:3]  # will return rgba, we take only first 3 so we get rgb
    color = matplotlib.colors.rgb2hex(rgb)
    return color
cl=['r','g','b','c','m','y','k']
color=dict()
for i,el in enumerate(cl):
    color.update({i:el})
#extract the labels for clutering precision its for after clustering
def time_convertor(x):
    s,h,m=0,0,0
    s=round(x%60,2)
    m=int(x/60)
    h=int(m/60)
    m=m%60
    time=str(h)+':'+str(m)+':'+str(s)
    return pd.to_datetime('1970-01-01 '+time)


# Display figures inline in Jupyter notebook
import seaborn as sns
# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(15, 5)})


In [3]:
#read data
with open("normalized_data.txt", "rb") as fp:
        dfRaw = pickle.load(fp)
        
def get_label(number,df):
    label_event_lenght=dict()
    df=pd.read_csv(f'{number}\\groundTruth.csv')
    df['length']=df.en-df.st
    for event , dft in df.groupby('evento'):
        if (event in label_event_lenght):
            label_event_lenght.update({event : label_event_lenght[event]+list(dft.length) })
        else:
            label_event_lenght.update({event : list(dft.length)})

    df['st_time']=df.apply(lambda x:time_convertor(x.st) ,axis=1 )
    df['en_time']=df.apply(lambda x:time_convertor(x.en) ,axis=1 )
    df=df.drop('st',1).drop('en',1).drop('length',1)
    evented_label=list()
    for i in range(len(df)):
        evented_label.append((df.iloc[i][0],df.iloc[i][1],df.iloc[i][2]))
    eve=pd.DataFrame(evented_label)
    eve[3]=[f'evented_label_{number}' for x in evented_label]
    return eve
Ev=[get_label(dfRaw[x][0],dfRaw[x][1]) for x in range(len(dfRaw))]
evented_label_All=pd.concat(Ev).reset_index().drop('index',1)

window={226:['curva_direita_agressiva','curva_esquerda_agressiva','aceleracao_agressiva'],
        126:['troca_faixa_direita_agressiva','troca_faixa_esquerda_agressiva'],
        186:['freada_agressiva']
       }


In [ ]:
#swich to thease data set for finding labels
with open("Data/data_list_per_driver", "rb") as fp:
        Sensory_data = pickle.load(fp)

Lambda=1
sample_rate=1#0.04/0.02
t1=time.time()
def TempEvent_genarator_new(Lambda,df,L,i):
    M=int((L)*Lambda)
    temp_event=list()
    pointer=0
    stop=False
    #go on timeserise for event extraction
    while stop!=True:
        clear_output(wait=True)
        pointer=pointer + M
        l_min=pointer-int(L/2)
        l_max=pointer+int(L/2)
        if l_max<0:
            l_max=L/2
        if (l_min<0):
            l_min=0
        if (l_max>len(df)):
            l_max=len(df)
            stop=True
        print('extraction ',l_max/len(df),' % of ',i,' is complete')
        temp_event.append(df[l_min:l_max])
    return temp_event

temp_event_test_new=list()
for l,DF in enumerate(Sensory_data):
    te=list()
    DF['g-x']=DF.apply(lambda x:(x[0]-min(DF['g-x'])/(max(DF['g-x'])-min(DF['g-x']))),axis=1)
    DF['g-y']=DF.apply(lambda x:(x[1]-min(DF['g-y'])/(max(DF['g-y'])-min(DF['g-y']))),axis=1)
    DF['g-z']=DF.apply(lambda x:(x[2]-min(DF['g-z'])/(max(DF['g-z'])-min(DF['g-z']))),axis=1)
    DF['a-x']=DF.apply(lambda x:(x[3]-min(DF['a-x'])/(max(DF['a-x'])-min(DF['a-x']))),axis=1)
    DF['a-y']=DF.apply(lambda x:(x[4]-min(DF['a-y'])/(max(DF['a-y'])-min(DF['a-y']))),axis=1)
    DF['a-z']=DF.apply(lambda x:(x[5]-min(DF['a-z'])/(max(DF['a-z'])-min(DF['a-z']))),axis=1)

    for i,lw in enumerate(window):
        te.append(TempEvent_genarator_new(Lambda,DF,lw/sample_rate,i+1))
    temp_event_test_new.append(te)

#make it dataframe
Data_test_new={
      226:[],
      126:[],
      186:[],
       }
for driver in range(len(temp_event_test_new)):
    Data_test_new[226]=Data_test_new[226]+[ x for x in temp_event_test_new[driver][0] ]
    Data_test_new[186]=Data_test_new[186]+[ x for x in temp_event_test_new[driver][1] ]
    Data_test_new[126]=Data_test_new[126]+[ x for x in temp_event_test_new[driver][2] ]
    
    
#calculate distance
k,n=0,0
num_event=dict(evented_label_All.groupby(0).size())
for lenght in window:
    for name in window[lenght]:
        k=k+len(Data_test_new[lenght])*num_event[name]

Data_DS_test_new={
      226:[],
      126:[],
      186:[],
       }

for wind in Data_test_new:
    for event in Data_test_new[wind]:
         #---------for each Data-distance from events----------
        Y=list()
        for from_event,dfe in evented_label_All.groupby(0):
            if from_event in window[wind]:
                X=list()
                #____for each event_distance from each label_____
                for el in dfe.iloc:
                    clear_output(wait=True)
                    n=n+1
                    print('disstance calculatoin ',round((n/k)*100,4),' %  is complete')
                    if   el[3]=='evented_label_16':
                        X.append(fastdtw(event,dfRaw[0][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_17':
                        X.append(fastdtw(event,dfRaw[1][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_20':
                        X.append(fastdtw(event,dfRaw[2][1][el[1]:el[2]])[0])
                    elif el[3]=='evented_label_21':
                        X.append(fastdtw(event,dfRaw[3][1][el[1]:el[2]])[0])
                #_________________________________________________
                Y.append((from_event,X))
        Data_DS_test_new[wind].append(Y)
        #------------------------------------------------------
#prepare dataset for labeling
Test_new=dict()
for win in Data_DS_test_new:
    dumy_list=list()
    for events in Data_DS_test_new[win]:
        dumy_dict=dict()
        for name , event in events:
            for tag,number in enumerate(event):
                dumy_dict.update({name+str(tag):number})
        dumy_list.append(dumy_dict)
    Test_new.update({win:pd.DataFrame(dumy_list)})
t2=time.time()
print(round((t2-t1)/60) ,'min has time left')

extraction  1.0  % of  3  is complete


In [ ]:
from copy import deepcopy
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator, export_text
from sklearn.metrics import accuracy_score, classification_report

with open("Model_new_1.txt", "rb") as fp:
    Result = pickle.load(fp)

In [ ]:
Max_depth=8
wid=186
6

estimator=Result[wid][Max_depth-1][1]

{x for x in estimator.predict(Test_new[wid])}

